# Evaluating Models

This notebook contains the code used for evaluating the following learning models:

-  **Standard GBDT** (_baseline 1_)
-  **Adversarial Boosting** (_baseline 2_)
-  **Non-Interferent GBDT** (our proposal)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import functools
from os import listdir
from os.path import isfile, join
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from nilib import *

# Standard evaluation metric

The following function is the one used for evaluating the quality of the learned model (either _standard_, _adversarial-boosting_, or _non-interferent_). This is the standard <code>avg_log_loss</code>.

In [3]:
def logistic(x):
    return 1.0/(1.0 + np.exp(-x))

In [4]:
def logit(p):
    return np.log(p/(1-p))

# <code>avg_log_loss</code>

In [5]:
# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False

In [6]:
def eval_log_loss(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), lgbm_test)[1]
    
    return avg_log_loss(model.predict(test.iloc[:,:-1].values), lgbm_test)[1]

# Custom evaluation metric

Similarly to what we have done for <code>fobj</code>, <code>feval</code> can be computed from a weighted combination of two evaluation metrics:

-  <code>avg_log_loss</code> (standard, defined above);
-  <code>avg_log_loss_uma</code> (custom, defined below).

# <code>avg_log_loss_uma</code>

This is the binary log loss yet modified to operate on groups of perturbed instances.

In [7]:
# Our custom metric

def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))
            
            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  
        
    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

In [8]:
def eval_log_loss_uma(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss_uma(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                               lgbm_test)[1]
    
    return avg_log_loss_uma(model.predict(test.iloc[:,:-1].values), 
                                               lgbm_test)[1]

# <code>feval=avg_non_interferent_log_loss</code>

Used for measuring the validity of any model (either _standard_, _baseline_, or _non-interferent_). More precisely, <code>avg_non_interferent_log_loss</code> is the weighted sum of the binary log loss and the binary log loss under maximal attack.

In [9]:
# LightGBM takes lambda x,y: avg_weighted_sum_log_loss_log_loss_uma(preds, train_data, alpha=0.5)

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    # _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    ids = []
    attack_lens = train_data.get_group()
    
    if attack_lens is not None:
        offset=0
        for atk in attack_lens:
            ids += [offset]
            offset += atk      
            
    ids = np.array(ids)
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds[ids]*labels[ids]))
    loss_plain = np.mean(losses)

    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={:.2f}]'.format(alpha), weighted_loss, False

def eval_non_interferent_log_loss(model, test, test_groups=None, svm=False, alpha=1.0):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_non_interferent_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]
    
    return avg_non_interferent_log_loss(model.predict(test.iloc[:,:-1].values), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]

# Additional validity measures

In addition to the evaluation metrics defined above (used for training), we also consider the following **4** measures of validity to compare the performance of each learned model:

-  <code>eval_binary_err_rate</code>: This is the traditional binary error rate (1-accuracy);
-  <code>eval_binary_err_rate_uma</code>: This is the binary error rate modified to operate on groups of perturbed instances under maximal attack.
-  <code>eval_roc_auc</code>: This is the classical ROC AUC score;
-  <code>eval_roc_auc_uma</code>: This is the ROC AUC score modified to operate on groups of perturbed instances under maximal attack.

Again, note that those are **not** metrics used at training time (i.e., they do not define any <code>feval</code>), rather they are used to assess the (offline) quality of each learned model.

# <code>eval_binary_err_rate</code>

In [10]:
def eval_binary_err_rate(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    errs = 0
    for p,l in zip(predictions,labels):
        if p != l:
            errs += 1
    return errs/len(predictions)

# <code>eval_binary_err_rate_uma</code>

In [11]:
def eval_binary_err_rate_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    errs = 0

    for g in test_groups:
        predictions_att = predictions[offset:offset+g]
        true_label = labels[offset]
        if np.any([p != true_label for p in predictions_att]):
            errs += 1
        offset += g

    return errs/len(test_groups)

# <code>eval_roc_auc</code>

In [12]:
def eval_roc_auc(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
        
    
    return roc_auc_score(y_true=labels, y_score=predictions)

# <code>eval_roc_auc_uma</code>

In [13]:
def eval_roc_auc_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
    
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return roc_auc_score(y_true=true_labels, y_score=worst_predictions)

# <code>eval_specificity</code>

In [14]:
def eval_specificity(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tn/(tn + fp)

# <code>eval_specificity_uma</code>

In [15]:
def eval_specificity_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tn/(tn + fp)

# <code>eval_precision</code>

In [16]:
def eval_precision(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tp/(tp + fp)

# <code>eval_precision_uma</code>

In [17]:
def eval_precision_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tp/(tp + fp)

# <code>eval_recall</code>

In [18]:
def eval_recall(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tp/(tp + fn)

# <code>eval_recall_uma</code>

In [19]:
def eval_recall_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tp/(tp + fn)

# <code>eval_f1</code>

In [20]:
def eval_f1(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    return f1_score(y_true=labels, y_pred=predictions, average='macro')

# <code>eval_f1_uma</code>

In [21]:
def eval_f1_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return f1_score(y_true=true_labels, y_pred=worst_predictions, average='macro')

# Evaluation metrics

In [22]:
EVAL_METRICS = [eval_log_loss, 
                eval_binary_err_rate,
                eval_specificity,
                eval_precision,
                eval_recall,
                eval_f1,
                eval_roc_auc
               ]

EVAL_METRICS_UNDER_MAX_ATTACK = [eval_log_loss_uma,
                                 eval_binary_err_rate_uma,
                                 eval_specificity_uma,
                                 eval_precision_uma,
                                 eval_recall_uma,
                                 eval_f1_uma,
                                 eval_roc_auc_uma
                                ]

# Evaluate each model w.r.t. _all_ evaluation metrics

In [23]:
def eval_learned_model(model, eval_metric, test, test_groups=None, svm=False):
    return eval_metric(model, test, test_groups=test_groups, svm=svm)

In [24]:
def eval_learned_models(model, model_type, test, test_groups=None, budget=0):

    eval_metrics = EVAL_METRICS
    d_test = "D_test"
    if test_groups is not None:
        eval_metrics = EVAL_METRICS_UNDER_MAX_ATTACK
        d_test = "D_test_att"
    
    header = ['Model'] + ['Budget'] + [m.__name__.replace('eval_','').replace('_',' ').strip().title() 
                                       for m in EVAL_METRICS]
    df = pd.DataFrame(columns=header)
    first_row = [model_type] + [budget] + [None for m in EVAL_METRICS]
    df.loc[0] = first_row
    
    svm = False
    if model_type == "SVM":
        svm = True

    for eval_metric in eval_metrics:
        res = eval_learned_model(model, eval_metric, test, test_groups=test_groups, svm=svm)
        print("{} learning - {} on {} = {:.5f}"
                  .format(model_type, eval_metric.__name__, d_test, res))
        column_metric = eval_metric.__name__
        if eval_metric.__name__.endswith("uma"):
            column_metric = eval_metric.__name__.replace('uma', '')
        df[column_metric.replace('eval_','').replace('_',' ').strip().title()] = res

    print("******************************************************************************************************")
    
    return df

# Path to files

In [25]:
DATASET_NAME="census" # census

In [26]:
DATASET_DIR="../data/{}".format(DATASET_NAME)
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
OUTPUT_FILENAME="../out/results/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train_ori.csv.bz2"
TRAINING_FILENAME_ATT=DATASET_DIR + "/" + "train_B{}.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid_ori.csv.bz2"
VALIDATION_FILENAME_ATT=DATASET_DIR + "/" + "valid_B{}.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test_ori.csv.bz2"
TEST_FILENAME_ATT=DATASET_DIR + "/" + "test_B{}.csv.bz2"

In [27]:
# TRAINING_BUDGETS = [20, 30, 40] # [5, 15, 150, 300]
TRAINING_BUDGETS= [5, 15, 150, 300] 

# Load plain datasets

In [28]:
# load train/valid/test (plain)
TRAIN, VALID, TEST, _ = load_atk_train_valid_test(TRAINING_FILENAME, VALIDATION_FILENAME, TEST_FILENAME)

Loading pre-processed files...


# Load attacked datasets

## Load an attacked dataset with a specific budget

In [29]:
def load_attacked_dataset(budget):
    # load train/valid/test (attacked)
    train_att, valid_att, test_att, _ = load_atk_train_valid_test(TRAINING_FILENAME_ATT.format(budget), 
                                                                  VALIDATION_FILENAME_ATT.format(budget), 
                                                                  TEST_FILENAME_ATT.format(budget))

    test_groups = test_att['instance_id'].value_counts().sort_index().values
    test_att = test_att.iloc[:, 1:]

    valid_groups = valid_att['instance_id'].value_counts().sort_index().values
    valid_att = valid_att.iloc[:, 1:]

    train_groups = train_att['instance_id'].value_counts().sort_index().values
    train_att = train_att.iloc[:, 1:]
    
    return train_att, train_groups, valid_att, valid_groups, test_att, test_groups

## Load _all_ the attacked datasets given a list of budgets

In [30]:
def load_attacked_datasets(budgets):
    att_datasets = {}
    for b in budgets:
        att_datasets[b] = load_attacked_dataset(b)
    
    return att_datasets

# Evaluate all models w.r.t. standard metrics (i.e., attack-free)

In [31]:
def extract_model_name(model_filename):
    model_fileroot = model_filename.split('/')[-1].split('.')[0]
    model_name = model_fileroot.split('_')[0].title()
    training_budget = ''
    budget = model_fileroot.split('_B')[-1].split('_')[0]
    try: 
        int(budget)
        training_budget = ' [train budget={}]'.format(budget)
    except:
        pass
    
    return model_name + training_budget

In [32]:
def eval_all_models(models_dir, test):
    
    model_csv = sorted(glob.glob(models_dir + "/*.csv"))
    model_filenames = []
    
    for m in model_csv:
        model_df = pd.read_csv(m)
        model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
    
    df = pd.concat([eval_learned_models(lightgbm.Booster(model_file=mf), 
                                        extract_model_name(mf), 
                                        test) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [33]:
eval_std_df = eval_all_models(MODELS_DIR, TEST)

Adv-Boosting [train budget=15] learning - eval_log_loss on D_test = 0.32143
Adv-Boosting [train budget=15] learning - eval_binary_err_rate on D_test = 0.14903
Adv-Boosting [train budget=15] learning - eval_specificity on D_test = 0.97056
Adv-Boosting [train budget=15] learning - eval_precision on D_test = 0.84186
Adv-Boosting [train budget=15] learning - eval_recall on D_test = 0.48263
Adv-Boosting [train budget=15] learning - eval_f1 on D_test = 0.76061
Adv-Boosting [train budget=15] learning - eval_roc_auc on D_test = 0.91200
******************************************************************************************************
Adv-Boosting [train budget=150] learning - eval_log_loss on D_test = 0.32209
Adv-Boosting [train budget=150] learning - eval_binary_err_rate on D_test = 0.14848
Adv-Boosting [train budget=150] learning - eval_specificity on D_test = 0.97203
Adv-Boosting [train budget=150] learning - eval_precision on D_test = 0.84793
Adv-Boosting [train budget=150] learning - e

Non-Interferent [train budget=300] learning - eval_precision on D_test = 0.75111
Non-Interferent [train budget=300] learning - eval_recall on D_test = 0.45873
Non-Interferent [train budget=300] learning - eval_f1 on D_test = 0.73186
Non-Interferent [train budget=300] learning - eval_roc_auc on D_test = 0.84157
******************************************************************************************************
Non-Interferent [train budget=5] learning - eval_log_loss on D_test = 0.66812
Non-Interferent [train budget=5] learning - eval_binary_err_rate on D_test = 0.16993
Non-Interferent [train budget=5] learning - eval_specificity on D_test = 0.95064
Non-Interferent [train budget=5] learning - eval_precision on D_test = 0.75111
Non-Interferent [train budget=5] learning - eval_recall on D_test = 0.45873
Non-Interferent [train budget=5] learning - eval_f1 on D_test = 0.73186
Non-Interferent [train budget=5] learning - eval_roc_auc on D_test = 0.84157
*************************************

In [34]:
eval_std_df

,Model,Budget,Log Loss,Binary Err Rate,Specificity,Precision,Recall,F1,Roc Auc
0,Adv-Boosting [train budget=15],0,0.321429,0.149033,0.970562,0.841857,0.482634,0.760608,0.912003
1,Adv-Boosting [train budget=150],0,0.322092,0.148480,0.972027,0.847930,0.480379,0.760712,0.912036
2,Adv-Boosting [train budget=300],0,0.321714,0.147927,0.972613,0.850758,0.480830,0.761445,0.912458
3,Adv-Boosting [train budget=5],0,0.320699,0.148811,0.969537,0.838384,0.486694,0.761794,0.912042
4,Non-Interferent [train budget=15],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569
5,Non-Interferent [train budget=150],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569
6,Non-Interferent [train budget=300],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569
7,Non-Interferent [train budget=5],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569
8,Non-Interferent [train budget=15],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569
9,Non-Interferent [train budget=150],0,0.668118,0.169928,0.950644,0.751108,0.458728,0.731864,0.841569


# Evaluate all models w.r.t. custom metrics (i.e., under attack)

In [35]:
att_datasets = load_attacked_datasets(TRAINING_BUDGETS)

Loading pre-processed files...
Loading pre-processed files...
Loading pre-processed files...
Loading pre-processed files...


In [36]:
def eval_all_models_under_attack_budget(models_dir, test, test_groups, budget):
    
    #model_filenames = sorted(glob.glob(models_dir + "/*.model"))
    
    model_csv = sorted(glob.glob(models_dir + "/*.csv"))
    model_filenames = []
    
    for m in model_csv:
        model_df = pd.read_csv(m)
        model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
    
    
    df = pd.concat([eval_learned_models(lightgbm.Booster(model_file=mf), 
                                        extract_model_name(mf), 
                                        test,
                                        test_groups, 
                                        budget=budget
                                       ) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [37]:
def eval_all_models_under_attack(models_dir, att_tests, budgets):
    
    eval_att_dfs = []

    for b in TRAINING_BUDGETS:
    
        eval_att_dfs.append(eval_all_models_under_attack_budget(models_dir, att_tests[b][4], att_tests[b][5], b))
        
        
    eval_att_df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Model', 'Budget']), eval_att_dfs)
    eval_att_df = pd.concat(eval_att_dfs, axis=0, sort=False)
    eval_att_df.reset_index(inplace=True, drop=True)
    
    return eval_att_df

In [ ]:
eval_att_df = eval_all_models_under_attack(MODELS_DIR, att_datasets, TRAINING_BUDGETS)

Adv-Boosting [train budget=15] learning - eval_log_loss_uma on D_test_att = 0.32330
Adv-Boosting [train budget=15] learning - eval_binary_err_rate_uma on D_test_att = 0.14981
Adv-Boosting [train budget=15] learning - eval_specificity_uma on D_test_att = 0.96954
Adv-Boosting [train budget=15] learning - eval_precision_uma on D_test_att = 0.83725
Adv-Boosting [train budget=15] learning - eval_recall_uma on D_test_att = 0.48263
Adv-Boosting [train budget=15] learning - eval_f1_uma on D_test_att = 0.75973
Adv-Boosting [train budget=15] learning - eval_roc_auc_uma on D_test_att = 0.91004
******************************************************************************************************
Adv-Boosting [train budget=150] learning - eval_log_loss_uma on D_test_att = 0.32395
Adv-Boosting [train budget=150] learning - eval_binary_err_rate_uma on D_test_att = 0.14947
Adv-Boosting [train budget=150] learning - eval_specificity_uma on D_test_att = 0.97071
Adv-Boosting [train budget=150] learning -

Non-Interferent [train budget=5] learning - eval_f1_uma on D_test_att = 0.71788
Non-Interferent [train budget=5] learning - eval_roc_auc_uma on D_test_att = 0.83061
******************************************************************************************************
Non-Interferent [train budget=15] learning - eval_log_loss_uma on D_test_att = 0.66881
Non-Interferent [train budget=15] learning - eval_binary_err_rate_uma on D_test_att = 0.18331
Non-Interferent [train budget=15] learning - eval_specificity_uma on D_test_att = 0.93292
Non-Interferent [train budget=15] learning - eval_precision_uma on D_test_att = 0.68949
Non-Interferent [train budget=15] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=15] learning - eval_f1_uma on D_test_att = 0.71788
Non-Interferent [train budget=15] learning - eval_roc_auc_uma on D_test_att = 0.83061
******************************************************************************************************
Non-Interferent [t

Non-Interferent [train budget=5] learning - eval_specificity_uma on D_test_att = 0.92897
Non-Interferent [train budget=5] learning - eval_precision_uma on D_test_att = 0.70853
Non-Interferent [train budget=5] learning - eval_recall_uma on D_test_att = 0.53180
Non-Interferent [train budget=5] learning - eval_f1_uma on D_test_att = 0.75019
Non-Interferent [train budget=5] learning - eval_roc_auc_uma on D_test_att = 0.87704
******************************************************************************************************
Std-Gbdt learning - eval_log_loss_uma on D_test_att = 0.32995
Std-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.15434
Std-Gbdt learning - eval_specificity_uma on D_test_att = 0.91974
Std-Gbdt learning - eval_precision_uma on D_test_att = 0.71414
Std-Gbdt learning - eval_recall_uma on D_test_att = 0.61750
Std-Gbdt learning - eval_f1_uma on D_test_att = 0.78114
Std-Gbdt learning - eval_roc_auc_uma on D_test_att = 0.89767
************************************

Non-Interferent [train budget=300] learning - eval_specificity_uma on D_test_att = 0.93014
Non-Interferent [train budget=300] learning - eval_precision_uma on D_test_att = 0.68072
Non-Interferent [train budget=300] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=300] learning - eval_f1_uma on D_test_att = 0.71574
Non-Interferent [train budget=300] learning - eval_roc_auc_uma on D_test_att = 0.82308
******************************************************************************************************
Non-Interferent [train budget=5] learning - eval_log_loss_uma on D_test_att = 0.66929
Non-Interferent [train budget=5] learning - eval_binary_err_rate_uma on D_test_att = 0.18541
Non-Interferent [train budget=5] learning - eval_specificity_uma on D_test_att = 0.93014
Non-Interferent [train budget=5] learning - eval_precision_uma on D_test_att = 0.68072
Non-Interferent [train budget=5] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train 

Non-Interferent [train budget=150] learning - eval_roc_auc_uma on D_test_att = 0.86546
******************************************************************************************************
Non-Interferent [train budget=300] learning - eval_log_loss_uma on D_test_att = 0.37617
Non-Interferent [train budget=300] learning - eval_binary_err_rate_uma on D_test_att = 0.17756
Non-Interferent [train budget=300] learning - eval_specificity_uma on D_test_att = 0.91330
Non-Interferent [train budget=300] learning - eval_precision_uma on D_test_att = 0.67019
Non-Interferent [train budget=300] learning - eval_recall_uma on D_test_att = 0.54263
Non-Interferent [train budget=300] learning - eval_f1_uma on D_test_att = 0.74281
Non-Interferent [train budget=300] learning - eval_roc_auc_uma on D_test_att = 0.86380
******************************************************************************************************
Non-Interferent [train budget=5] learning - eval_log_loss_uma on D_test_att = 0.36834
Non

Non-Interferent [train budget=15] learning - eval_f1_uma on D_test_att = 0.70606
Non-Interferent [train budget=15] learning - eval_roc_auc_uma on D_test_att = 0.81628
******************************************************************************************************
Non-Interferent [train budget=150] learning - eval_log_loss_uma on D_test_att = 0.66981
Non-Interferent [train budget=150] learning - eval_binary_err_rate_uma on D_test_att = 0.19502
Non-Interferent [train budget=150] learning - eval_specificity_uma on D_test_att = 0.91740
Non-Interferent [train budget=150] learning - eval_precision_uma on D_test_att = 0.64326
Non-Interferent [train budget=150] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=150] learning - eval_f1_uma on D_test_att = 0.70606
Non-Interferent [train budget=150] learning - eval_roc_auc_uma on D_test_att = 0.81628
******************************************************************************************************
Non-Inter

Non-Interferent [train budget=15] learning - eval_binary_err_rate_uma on D_test_att = 0.18143
Non-Interferent [train budget=15] learning - eval_specificity_uma on D_test_att = 0.91139
Non-Interferent [train budget=15] learning - eval_precision_uma on D_test_att = 0.66125
Non-Interferent [train budget=15] learning - eval_recall_uma on D_test_att = 0.53270
Non-Interferent [train budget=15] learning - eval_f1_uma on D_test_att = 0.73678
Non-Interferent [train budget=15] learning - eval_roc_auc_uma on D_test_att = 0.86000
******************************************************************************************************
Non-Interferent [train budget=150] learning - eval_log_loss_uma on D_test_att = 0.39438
Non-Interferent [train budget=150] learning - eval_binary_err_rate_uma on D_test_att = 0.18574
Non-Interferent [train budget=150] learning - eval_specificity_uma on D_test_att = 0.90554
Non-Interferent [train budget=150] learning - eval_precision_uma on D_test_att = 0.64696
Non-Interf

Non-Interferent [train budget=5] learning - eval_log_loss_uma on D_test_att = 0.67040
Non-Interferent [train budget=5] learning - eval_binary_err_rate_uma on D_test_att = 0.20343
Non-Interferent [train budget=5] learning - eval_specificity_uma on D_test_att = 0.90627
Non-Interferent [train budget=5] learning - eval_precision_uma on D_test_att = 0.61376
Non-Interferent [train budget=5] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=5] learning - eval_f1_uma on D_test_att = 0.69780
Non-Interferent [train budget=5] learning - eval_roc_auc_uma on D_test_att = 0.80905
******************************************************************************************************
Non-Interferent [train budget=15] learning - eval_log_loss_uma on D_test_att = 0.67040
Non-Interferent [train budget=15] learning - eval_binary_err_rate_uma on D_test_att = 0.20343
Non-Interferent [train budget=15] learning - eval_specificity_uma on D_test_att = 0.90627
Non-Interferent [train

Non-Interferent [train budget=300] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=300] learning - eval_f1_uma on D_test_att = 0.69780
Non-Interferent [train budget=300] learning - eval_roc_auc_uma on D_test_att = 0.80905
******************************************************************************************************
Non-Interferent [train budget=5] learning - eval_log_loss_uma on D_test_att = 0.67040
Non-Interferent [train budget=5] learning - eval_binary_err_rate_uma on D_test_att = 0.20343
Non-Interferent [train budget=5] learning - eval_specificity_uma on D_test_att = 0.90627
Non-Interferent [train budget=5] learning - eval_precision_uma on D_test_att = 0.61376
Non-Interferent [train budget=5] learning - eval_recall_uma on D_test_att = 0.45873
Non-Interferent [train budget=5] learning - eval_f1_uma on D_test_att = 0.69780
Non-Interferent [train budget=5] learning - eval_roc_auc_uma on D_test_att = 0.80905
**************************************

In [ ]:
eval_att_df.head()

In [ ]:
overall_df = pd.concat([eval_std_df, eval_att_df], 
                       axis=0, 
                       sort=False)
overall_df.reset_index(inplace=True, drop=True)

In [ ]:
overall_df

# Save the DataFrame containing results

In [ ]:
overall_df.to_csv(OUTPUT_FILENAME + ".csv", sep=",", index=False)

In [ ]:
overall_df[ overall_df['Model'].str.contains("=300") | overall_df['Model'].str.contains("Gbdt")]

In [ ]:
att_datasets[5][0].capital_gain.value_counts()